In [1]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

### Create Dataframe of Neighbourhoods

In [2]:
from bs4 import BeautifulSoup

In [3]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [5]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df4 = df3[df3.Borough != 'Not assigned']
df5 = df4.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df6 = df5.replace("Not assigned", "Queen's Park")
df6.shape

(103, 3)

In [6]:
df6.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### Obtain Neighbourhood Coordinates

In [7]:
df_coord = pd.read_csv('https://cocl.us/Geospatial_data')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_neighbourhoods = pd.merge(df6, df_coord, how='left',
        left_on='Postcode', right_on='Postal Code', validate="1:1")
df_neighbourhoods.drop(labels='Postal Code', axis=1, inplace=True)
df_neighbourhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [9]:
CLIENT_ID = 'F3UHLSQQ3BX0WQREMP10OWFSV3NZDYXQESM342ZGSRSEE2LV' # your Foursquare ID
CLIENT_SECRET = '53LF3MCDLIPWT4BFQCE2JXCRJPNXP4QRAZBNB0QDBSSIFR4Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F3UHLSQQ3BX0WQREMP10OWFSV3NZDYXQESM342ZGSRSEE2LV
CLIENT_SECRET:53LF3MCDLIPWT4BFQCE2JXCRJPNXP4QRAZBNB0QDBSSIFR4Y


In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [11]:
categoryId = '4d4b7105d754a06372d81259'
NEAR = 'Toronto,ON'
print(categoryId + " near " + NEAR + ' .... OK!')

4d4b7105d754a06372d81259 near Toronto,ON .... OK!


In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&near={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, NEAR, categoryId)
url

'https://api.foursquare.com/v2/venues/search?client_id=F3UHLSQQ3BX0WQREMP10OWFSV3NZDYXQESM342ZGSRSEE2LV&client_secret=53LF3MCDLIPWT4BFQCE2JXCRJPNXP4QRAZBNB0QDBSSIFR4Y&v=20180605&near=Toronto,ON&categoryId=4d4b7105d754a06372d81259'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd326db618f43001b05653d'},
 'response': {'venues': [{'id': '4adcc6ecf964a520b02f21e3',
    'name': 'Humber College - North Campus',
    'location': {'address': '205 Humber College Blvd.',
     'crossStreet': 'at Hwy 27',
     'lat': 43.72886399890596,
     'lng': -79.60704634474938,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.72886399890596,
       'lng': -79.60704634474938}],
     'postalCode': 'M9W 5L7',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['205 Humber College Blvd. (at Hwy 27)',
      'Toronto ON M9W 5L7',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1a2941735',
      'name': 'Community College',
      'pluralName': 'Community Colleges',
      'shortName': 'Community College',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/communitycollege_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 

In [14]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,hierarchy,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1a2941735', 'name': 'C...",False,NaN,4adcc6ecf964a520b02f21e3,205 Humber College Blvd.,CA,Toronto,Canada,at Hwy 27,"[205 Humber College Blvd. (at Hwy 27), Toronto...","[{'label': 'display', 'lat': 43.72886399890596...",43.728864,-79.607046,M9W 5L7,ON,Humber College - North Campus,v-1574119129,NaN
1,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",False,NaN,4ae0c938f964a520758221e3,105 St. George St.,CA,Toronto,Canada,University of Toronto,"[105 St. George St. (University of Toronto), T...","[{'label': 'display', 'lat': 43.6647972002443,...",43.664797,-79.398576,M5S 3E6,ON,Rotman School of Management,v-1574119129,NaN
2,"[{'id': '4bf58dd8d48988d1a2941735', 'name': 'C...",False,NaN,4afd5679f964a5205d2722e3,1750 Finch Ave E,CA,Toronto,Canada,at Seneca Hill Dr,"[1750 Finch Ave E (at Seneca Hill Dr), Toronto...","[{'label': 'display', 'lat': 43.79563252135708...",43.795633,-79.349484,M2J 2X5,ON,Seneca College Newnham Campus,v-1574119129,NaN
3,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,[],4adfae8df964a520b87c21e3,9 Kings College Circle,CA,Toronto,Canada,University of Toronto,[9 Kings College Circle (University of Toronto...,"[{'label': 'display', 'lat': 43.66237774900301...",43.662378,-79.394006,M5S 1A5,ON,Gerstein Science Information Centre,v-1574119129,NaN
4,"[{'id': '4bf58dd8d48988d19e941735', 'name': 'C...",False,[],5702c6e6cd10e91591b00f1c,4700 Keele St,CA,Toronto,Canada,NaN,"[4700 Keele St, Toronto ON M3J 1P3, Canada]","[{'label': 'display', 'lat': 43.77215834304510...",43.772158,-79.506450,M3J 1P3,ON,Bergeron Centre for Engineeering Excellence,v-1574119129,NaN


In [15]:
dataframe.shape

(30, 18)

In [16]:
df_college = dataframe.drop(columns=['categories', 'hierarchy','hasPerk', 'referralId', 'venuePage.id'])

In [17]:
df_college

,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name
0,4adcc6ecf964a520b02f21e3,205 Humber College Blvd.,CA,Toronto,Canada,at Hwy 27,"[205 Humber College Blvd. (at Hwy 27), Toronto...","[{'label': 'display', 'lat': 43.72886399890596...",43.728864,-79.607046,M9W 5L7,ON,Humber College - North Campus
1,4ae0c938f964a520758221e3,105 St. George St.,CA,Toronto,Canada,University of Toronto,"[105 St. George St. (University of Toronto), T...","[{'label': 'display', 'lat': 43.6647972002443,...",43.664797,-79.398576,M5S 3E6,ON,Rotman School of Management
2,4afd5679f964a5205d2722e3,1750 Finch Ave E,CA,Toronto,Canada,at Seneca Hill Dr,"[1750 Finch Ave E (at Seneca Hill Dr), Toronto...","[{'label': 'display', 'lat': 43.79563252135708...",43.795633,-79.349484,M2J 2X5,ON,Seneca College Newnham Campus
3,4adfae8df964a520b87c21e3,9 Kings College Circle,CA,Toronto,Canada,University of Toronto,[9 Kings College Circle (University of Toronto...,"[{'label': 'display', 'lat': 43.66237774900301...",43.662378,-79.394006,M5S 1A5,ON,Gerstein Science Information Centre
4,5702c6e6cd10e91591b00f1c,4700 Keele St,CA,Toronto,Canada,NaN,"[4700 Keele St, Toronto ON M3J 1P3, Canada]","[{'label': 'display', 'lat': 43.77215834304510...",43.772158,-79.506450,M3J 1P3,ON,Bergeron Centre for Engineeering Excellence
5,4ae4bcd1f964a520969d21e3,350 Victoria St.,CA,Toronto,Canada,at Ryerson University,"[350 Victoria St. (at Ryerson University), Tor...","[{'label': 'display', 'lat': 43.65787029408339...",43.657870,-79.380309,M5B 2K3,ON,Ryerson University Library
6,50292c47e4b0ca628adc33c6,50 Carlton St.,CA,Toronto,Canada,at Church St.,"[50 Carlton St. (at Church St.), Toronto ON M5...","[{'label': 'display', 'lat': 43.66175204715793...",43.661752,-79.380137,M5B 2A1,ON,Mattamy Athletic Centre
7,4ad78121f964a5208b0b21e3,4700 Keele St.,CA,Toronto,Canada,NaN,"[4700 Keele St., Toronto ON M3J 1P3, Canada]","[{'label': 'display', 'lat': 43.77210701043441...",43.772107,-79.502190,M3J 1P3,ON,York University - Keele Campus
8,4adf60eff964a5200e7a21e3,70 The Pond Rd.,CA,Toronto,Canada,NaN,"[70 The Pond Rd., Toronto ON M3J 3M6, Canada]","[{'label': 'display', 'lat': 43.77153051642339...",43.771531,-79.499497,M3J 3M6,ON,Seneca@York Campus
9,4af20df8f964a52086e521e3,100 St. George St,CA,Toronto,Canada,University of Toronto,"[100 St. George St (University of Toronto), To...","[{'label': 'display', 'lat': 43.66258359953375...",43.662584,-79.398588,NaN,ON,Sidney Smith Hall


In [18]:
df_college['Postcode'] = df_college['location.postalCode'].str[:3]

In [19]:
df_college.head()

,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,Postcode
0,4adcc6ecf964a520b02f21e3,205 Humber College Blvd.,CA,Toronto,Canada,at Hwy 27,"[205 Humber College Blvd. (at Hwy 27), Toronto...","[{'label': 'display', 'lat': 43.72886399890596...",43.728864,-79.607046,M9W 5L7,ON,Humber College - North Campus,M9W
1,4ae0c938f964a520758221e3,105 St. George St.,CA,Toronto,Canada,University of Toronto,"[105 St. George St. (University of Toronto), T...","[{'label': 'display', 'lat': 43.6647972002443,...",43.664797,-79.398576,M5S 3E6,ON,Rotman School of Management,M5S
2,4afd5679f964a5205d2722e3,1750 Finch Ave E,CA,Toronto,Canada,at Seneca Hill Dr,"[1750 Finch Ave E (at Seneca Hill Dr), Toronto...","[{'label': 'display', 'lat': 43.79563252135708...",43.795633,-79.349484,M2J 2X5,ON,Seneca College Newnham Campus,M2J
3,4adfae8df964a520b87c21e3,9 Kings College Circle,CA,Toronto,Canada,University of Toronto,[9 Kings College Circle (University of Toronto...,"[{'label': 'display', 'lat': 43.66237774900301...",43.662378,-79.394006,M5S 1A5,ON,Gerstein Science Information Centre,M5S
4,5702c6e6cd10e91591b00f1c,4700 Keele St,CA,Toronto,Canada,NaN,"[4700 Keele St, Toronto ON M3J 1P3, Canada]","[{'label': 'display', 'lat': 43.77215834304510...",43.772158,-79.506450,M3J 1P3,ON,Bergeron Centre for Engineeering Excellence,M3J


In [20]:
df_college2 = pd.merge(df_college, df_neighbourhoods, how='left',
        left_on='Postcode', right_on='Postcode', validate="m:m")
df_college2.head()

,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,4adcc6ecf964a520b02f21e3,205 Humber College Blvd.,CA,Toronto,Canada,at Hwy 27,"[205 Humber College Blvd. (at Hwy 27), Toronto...","[{'label': 'display', 'lat': 43.72886399890596...",43.728864,-79.607046,M9W 5L7,ON,Humber College - North Campus,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,4ae0c938f964a520758221e3,105 St. George St.,CA,Toronto,Canada,University of Toronto,"[105 St. George St. (University of Toronto), T...","[{'label': 'display', 'lat': 43.6647972002443,...",43.664797,-79.398576,M5S 3E6,ON,Rotman School of Management,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
2,4afd5679f964a5205d2722e3,1750 Finch Ave E,CA,Toronto,Canada,at Seneca Hill Dr,"[1750 Finch Ave E (at Seneca Hill Dr), Toronto...","[{'label': 'display', 'lat': 43.79563252135708...",43.795633,-79.349484,M2J 2X5,ON,Seneca College Newnham Campus,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
3,4adfae8df964a520b87c21e3,9 Kings College Circle,CA,Toronto,Canada,University of Toronto,[9 Kings College Circle (University of Toronto...,"[{'label': 'display', 'lat': 43.66237774900301...",43.662378,-79.394006,M5S 1A5,ON,Gerstein Science Information Centre,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
4,5702c6e6cd10e91591b00f1c,4700 Keele St,CA,Toronto,Canada,NaN,"[4700 Keele St, Toronto ON M3J 1P3, Canada]","[{'label': 'display', 'lat': 43.77215834304510...",43.772158,-79.506450,M3J 1P3,ON,Bergeron Centre for Engineeering Excellence,M3J,North York,"Northwood Park,York University",43.767980,-79.487262


In [21]:
df_colneigh = df_college2[['name', 'Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']].copy()

In [83]:
df_colneigh

,name,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,Humber College - North Campus,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,Rotman School of Management,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
2,Seneca College Newnham Campus,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
3,Gerstein Science Information Centre,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
4,Bergeron Centre for Engineeering Excellence,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
5,Ryerson University Library,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
6,Mattamy Athletic Centre,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
7,York University - Keele Campus,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
8,Seneca@York Campus,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
9,Sidney Smith Hall,NaN,NaN,NaN,NaN,NaN


,name,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,Humber College - North Campus,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,Rotman School of Management,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
2,Seneca College Newnham Campus,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
4,Bergeron Centre for Engineeering Excellence,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
5,Ryerson University Library,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
6,Mattamy Athletic Centre,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
7,York University - Keele Campus,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
8,Seneca@York Campus,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
9,Sidney Smith Hall,NaN,NaN,NaN,NaN,NaN
11,Robarts Library,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049


In [38]:
df_neighcount = df_colneigh.groupby(['Neighbourhood']).count()

df_neighcount

,name,Postcode,Borough,Latitude,Longitude
Neighbourhood,,,,,
Agincourt,1,1,1,1,1
Central Bay Street,1,1,1,1,1
"Fairview,Henry Farm,Oriole",2,2,2,2,2
"Harbord,University of Toronto",7,7,7,7,7
"Highland Creek,Rouge Hill,Port Union",1,1,1,1,1
"Humber Bay Shores,Mimico South,New Toronto",1,1,1,1,1
Northwest,1,1,1,1,1
"Northwood Park,York University",4,4,4,4,4
Rosedale,1,1,1,1,1


In [52]:
df_compare = pd.merge(df_neighbourhoods, df_neighcount, on=['Neighbourhood'], how='outer')   
    
df_compare

,Postcode_x,Borough_x,Neighbourhood,Latitude_x,Longitude_x,name,Postcode_y,Borough_y,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,43.753259,-79.329656,NaN,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,NaN,NaN,NaN,NaN,NaN
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,NaN,NaN,NaN,NaN,NaN
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,NaN,NaN,NaN,NaN,NaN
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN
7,M3B,North York,Don Mills North,43.745906,-79.352188,NaN,NaN,NaN,NaN,NaN
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,NaN,NaN,NaN,NaN,NaN
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,3.0,3.0,3.0,3.0,3.0


In [59]:
df_compare = df_compare.sort_values(by=['name'])

df_compare.head(12)

,Postcode_x,Borough_x,Neighbourhood,Latitude_x,Longitude_x,name,Postcode_y,Borough_y,Latitude_y,Longitude_y
12,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,1.0,1.0,1.0,1.0,1.0
22,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,1.0,1.0,1.0,1.0
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1.0,1.0,1.0,1.0,1.0
74,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,1.0,1.0,1.0,1.0,1.0
78,M1S,Scarborough,Agincourt,43.794200,-79.262029,1.0,1.0,1.0,1.0,1.0
88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,1.0,1.0,1.0,1.0,1.0
91,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1.0,1.0,1.0,1.0,1.0
94,M9W,Etobicoke,Northwest,43.706748,-79.594054,1.0,1.0,1.0,1.0,1.0
33,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,2.0,2.0,2.0,2.0,2.0
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,3.0,3.0,3.0,3.0,3.0


In [78]:
df_compare = df_compare.fillna(0)

In [79]:
df_none = df_compare[df_compare.name < 1]

df_none

,Postcode_x,Borough_x,Neighbourhood,Latitude_x,Longitude_x,name,Postcode_y,Borough_y,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,0.0,0.0,0.0,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,0.0,0.0,0.0,0.0
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,0.0,0.0,0.0,0.0
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,0.0,0.0,0.0,0.0
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,0.0,0.0,0.0,0.0
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242,0.0,0.0,0.0,0.0,0.0
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,0.0,0.0,0.0,0.0
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,0.0,0.0,0.0,0.0
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0.0,0.0,0.0,0.0,0.0
10,M6B,North York,Glencairn,43.709577,-79.445073,0.0,0.0,0.0,0.0,0.0


In [82]:
df_neighbourhoods

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [84]:
df_colneigh

,name,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,Humber College - North Campus,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,Rotman School of Management,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
2,Seneca College Newnham Campus,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
3,Gerstein Science Information Centre,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049
4,Bergeron Centre for Engineeering Excellence,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
5,Ryerson University Library,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
6,Mattamy Athletic Centre,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
7,York University - Keele Campus,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
8,Seneca@York Campus,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
9,Sidney Smith Hall,NaN,NaN,NaN,NaN,NaN


In [93]:
df_colneigh = df_colneigh.fillna(0)

In [94]:
# create map
school_map = folium.Map(location=[latitude, longitude], zoom_start=11)
markers_colors = []
for lat, lon in zip(df_colneigh['Latitude'], df_colneigh['Longitude']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        fill=True,
        fill_opacity=0.7).add_to(school_map)
school_map
